In [ ]:

import re
import pickle
from collections import Counter

# Set up OpenAI API key
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

import random

# Set the seed for reproducibility
random.seed(42)


In [ ]:
from huggingface_hub import login
login("")
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import pickle
from collections import Counter
from openai import OpenAI
from mistralai import Mistral
# Set up OpenAI API key
client_openai=  OpenAI(api_key="",
)

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

client_mistral = Mistral(api_key="5O")

import random

# Set the seed for reproducibility
random.seed(42)


In [ ]:


# system_prompt_nepali = "तपाईं एक उत्कृष्ट भाषाविज्ञ हुनुहुन्छ। "

def generate_NER(model_name,prompt):


    generated_text=""

    if model_name=="openai":
        completion = client_openai.chat.completions.create(
            model = 'gpt-4o',
            messages = [
                # {'role':'system',"content": "You are an excellent linguist. "},
            #   {'role':'system',"content": "तपाईं एक उत्कृष्ट भाषाविज्ञ हुनुहुन्छ। "},
                {'role': 'user', 'content':prompt}
            ],
            # temperature = 0  ,
                max_tokens=500,
                temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  n=1
            )

        generated_text = completion.choices[0].message.content.strip()

    

    return generated_text


In [ ]:
entities_list=["Location","Date","Person","Organization","Event"]

In [ ]:
def parse_file(file_path):
  with open(file_path,"r") as f:
    tokens,labels = [],[]
    t,l = [], []
    for line in f.readlines():
      tmp = line.strip().split()
      if len(tmp) == 0:
        tokens.append(t)
        labels.append(l)
        t, l = [], []
      else:
        t.append(tmp[0])
        l.append(tmp[1])
    if len(t) > 0:
      tokens.append(t)
      labels.append(l)
    data = tokens,labels
    return data

def get_news_data_sets():
  train_data= parse_file("everest-ner/EverestNER-train-bio.txt")
  test_data= parse_file("everest-ner/EverestNER-test-bio.txt")
  return train_data,test_data

def get_tweets_data_sets():
  train_data = parse_file("DanfeNER/DanfeNER-train-bio.txt")
  test_data = parse_file("DanfeNER/DanfeNER-test-bio.txt")
  return train_data,test_data



In [ ]:
news_train,news_test=get_news_data_sets()
news_train_sentences, news_train_labels = news_train
news_test_sentences, news_test_labels = news_test
len(news_train_sentences),len(news_test_sentences)

In [ ]:
def convert_to_bio(entity, prediction,original_sentence):
    tokens = prediction.split()
    original_tokens=original_sentence.split()
    bio_labels = []
    inside_entity = False  # Tracks if we are inside an entity

    for token in tokens:
        if '@@' in token and '##' in token:  # Entire entity in one token
            bio_labels.append(f"B-{entity}")
            inside_entity = False
        elif '@@' in token:  # Entity begins in this token
            bio_labels.append(f"B-{entity}")
            inside_entity = True
        elif '##' in token:  # Entity ends in this token
            bio_labels.append(f"I-{entity}")
            inside_entity = False
        else:
            if inside_entity:  # Continuation of the entity
                bio_labels.append(f"I-{entity}")
            else:  # Outside of any entity
                bio_labels.append("O")
    if len(bio_labels)==0:
        bio_labels=["O"]*len(original_tokens)

    return bio_labels

In [ ]:
def align_sentences(S, T):
    """
    Aligns the predicted sentence T to match the reference sentence S
    while preserving special tokens ('@@' and '##') in T.

    Parameters:
    S (str): The reference sentence.
    T (str): The predicted sentence.

    Returns:
    str: The aligned version of T, matching the structure of S.
    """
    # Split sentences into tokens
    s_tokens = S.split()
    t_tokens = T.split()

    aligned_t_tokens = []
    t_index = 0  # Pointer for T tokens

    for s_token in s_tokens:
        if t_index < len(t_tokens):
            t_token = t_tokens[t_index]

            # If the current token in T contains special markers, preserve it.
            if '@@' in t_token or '##' in t_token:
                aligned_t_tokens.append(t_token)
                t_index += 1  # Move to the next token in T
            else:
                # Align tokens from T to match S
                if t_token == s_token:
                    aligned_t_tokens.append(t_token)
                else:
                    aligned_t_tokens.append(s_token)
                t_index += 1
        else:
            # If T is shorter than S, pad with tokens from S
            aligned_t_tokens.append(s_token)

    # Handle any remaining tokens in T after exhausting S
    while t_index < len(t_tokens):
        aligned_t_tokens.append(t_tokens[t_index])
        t_index += 1

    # Join aligned tokens back into a sentence
    return ' '.join(aligned_t_tokens)

In [ ]:
def post_process_output(output):
    prediction = output
    
    # Check if "Output:" is in the prediction and process accordingly
    if "Output:" in prediction:
        prediction = prediction.split("Output:")[1].strip()

    if "नतिजा:" in prediction:
        prediction = prediction.split("नतिजा:")[1].strip()

    if "वाक्य:" in prediction:
        prediction = prediction.split("वाक्य:")[1].strip()
    
    # Extract portion before "Note" if it exists
    if "Note" in prediction:
        prediction = prediction.split("Note", 1)[0].strip()
    
    # Extract up to the first occurrence of "।"
    if "।" in prediction:
        prediction = prediction.split("।", 1)[0] + "।"
    
    # Return the processed prediction
    return prediction

In [ ]:
with open("output_path/train_datasets_with_tagging.pkl", "rb") as file:  # "rb" stands for read binary
    train_datasets_with_tagging = pickle.load(file)

with open("output_path/test_datasets_with_tagging.pkl", "rb") as file:  # "rb" stands for read binary
    test_datasets_with_tagging = pickle.load(file)


In [ ]:
ground_truth_separated={}
for sentence in test_datasets_with_tagging:
    # if sentence == "\' माइती टाढा ।":
        gd=test_datasets_with_tagging[sentence][1]
        a={}
        for ent in entities_list:
            temp=["O"]*len(gd)
            pos=[index for index, label in enumerate(gd) if ent in label]
            if len(pos)>0:
                for i in pos:
                    temp[i]=gd[i]
            # if len(temp)!=len(gd):
                # print("I")
            a[ent]=temp
        ground_truth_separated[sentence]=a

In [ ]:
# ground_truth_separated

In [ ]:
output_file_name="k_10_semantic_NEP.pkl"

with open("output_path/"+output_file_name , "rb") as file:  # "rb" stands for read binary
    k_10_semantic_NEP = pickle.load(file)

In [ ]:
import random

def select_random_keys(input_dict, num_keys=100, seed=42):
    """
    Selects a specified number of keys randomly from a dictionary.
    
    Args:
        input_dict (dict): The dictionary to sample keys from.
        num_keys (int): The number of keys to select.
        seed (int): The seed for random number generator.
    
    Returns:
        list: A list of randomly selected keys.
    """
    random.seed(seed)  # Set the seed for reproducibility
    all_keys = list(input_dict.keys())
    
    # Ensure the number of keys does not exceed the available keys
    if num_keys > len(all_keys):
        raise ValueError("num_keys exceeds the number of available keys in the dictionary.")
    
    return random.sample(all_keys, num_keys)

 # Example dictionary with 500 keys
output_file_name=k_10_semantic_NEP
random_keys = select_random_keys(output_file_name, num_keys=100, seed=123)

print(random_keys)

In [ ]:
def validate_predictions(sentence, ground_truths, predictions):
    updated_predictions = {}
    all_prompts={}
    # Iterate over each entity type in the predictions
    for entity_type, prediction in predictions.items():
        ground_truth = ground_truths.get(entity_type, [])
        updated_predictions[entity_type] = ["O"] * len(prediction)
        tokens = sentence.split()

        i = 0
        while i < len(prediction):
            if prediction[i].startswith("B-"):
                # Start of a new entity span
                entity_span = [tokens[i]]
                entity_indices = [i]
                entity_label = prediction[i][2:]  # Extract the label type

                # Collect continuation tokens
                j = i + 1
                while j < len(prediction) and prediction[j] == f"I-{entity_label}":
                    entity_span.append(tokens[j])
                    entity_indices.append(j)
                    j += 1

                # Validate the entire entity span
                entity_value = " ".join(entity_span)

                # Prompt GPT to validate
                # prompt = f"Is '{entity_value}' a {entity_type} in this sentence: '{sentence}'? Please answer with Yes or No. No explanation is needed."
                if len(entity_span)>1:
                    prompt=f"The given sentence: {sentence} \nAre the words {entity_value} in the given sentence a {entity_type} entity? Please answer with Yes or No. No explanation is needed. "
                else:
                    prompt=f"The given sentence: {sentence} \nIs the word {entity_value} in the given sentence a {entity_type} entity? Please answer with Yes or No. No explanation is needed. "
                # print(prompt)
                
                
                try:
                    answer=generate_NER("openai",prompt)
                    # print(answer)
                    all_prompts[prompt]=answer
                    # Update the prediction based on GPT's response
                    if "Yes" in answer:
                        # print("I came here")
                        for idx in entity_indices:
                            updated_predictions[entity_type][idx] = prediction[idx]
                    
                except Exception as e:
                    print(f"Error querying OpenAI: {e}")
                    # updated_predictions[entity_type].append("O")
            # else:
            #     updated_predictions[entity_type].append("O")
            i+=1
    return updated_predictions,all_prompts

In [ ]:
y_true1=[]
y_pred1=[]
y_pred2=[]
count=0
error_sentences={}
count=0
self_corrections={}
original_predictions={}
new_predictions={}
for sentence in random_keys[:]:
    true_label=ground_truth_separated[sentence]
    temp={}
    temp1={}
    temp2={}
    for entity in entities_list:

        prediction=(output_file_name[sentence][entity])
        prediction=post_process_output(prediction)

        predicted_sentences=convert_to_bio(entity,prediction,sentence)
       
        if len(predicted_sentences)==len(true_label[entity]):

            y_true1.append(true_label[entity])
            y_pred1.append(predicted_sentences)
            
            validated_predictions,all_prompts = validate_predictions(sentence, {entity:true_label[entity]}, {entity:predicted_sentences})
            y_pred2.append(validated_predictions[entity])
            temp1[entity]=predicted_sentences
            temp2[entity]=validated_predictions
            temp[entity]=all_prompts
    self_corrections[sentence]=temp
    original_predictions[sentence]=temp1
    new_predictions[sentence]=temp2

In [ ]:
print(classification_report(y_true1,y_pred1, zero_division=1))

In [ ]:
print(classification_report(y_true1,y_pred2, zero_division=1))